In [19]:
import json
import urllib
import pandas as pd

In [22]:
url = "paste neuroglancer url here, do not copy from share button"
csv_file_path = "annotations.csv"

url_encoded_json = url.split('#')[-1]
json_data = json.loads(urllib.parse.unquote(url_encoded_json[1:]))

annotations = []
for layer in json_data["layers"]:
    if layer["type"] == "annotation":
        layer_name = layer['name']
        
        for annotation in layer["annotations"]:
            annotation_type = annotation.get("type")
            # annotation_id = annotation.get("id")
            point = annotation.get("point")
            pointA = annotation.get("pointA")
            pointB = annotation.get("pointB")
            center = annotation.get("center")
            radii = annotation.get("radii")

            if point:
                annotations.append({
                    "layer_name": layer_name,
                    # "id": annotation_id,
                    "type": annotation_type,
                    "point": point
                })
            elif pointA and pointB:
                annotations.append({
                    "layer_name": layer_name,
                    # "id": annotation_id,
                    "type": annotation_type,
                    "point": (pointA, pointB)  # Store both points for AABB
                })
            elif annotation_type == 'line' and pointA and pointB:
                annotations.append({
                    "layer_name": layer_name,
                    # "id": annotation_id,
                    "type": annotation_type,
                    "point": (pointA, pointB)
                })
            elif annotation_type == 'ellipsoid' and center and radii:
                annotations.append({
                    "layer_name": layer_name,
                    # "id": annotation_id,
                    "type": annotation_type,
                    "point": (center, radii) #Ellipsoid centered at center with radii
                })

# Display extracted annotations
for annotation in annotations:
    print(annotation)

# Create a DataFrame from the annotations
annotations_df = pd.DataFrame(annotations)

# Format points for better readability in CSV
annotations_df["point"] = annotations_df["point"].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else f"{x[0][0]}, {x[0][1]}, {x[0][2]} - {x[1][0]}, {x[1][1]}, {x[1][2]}")

# Export to CSV
annotations_df.to_csv(csv_file_path, index=False)

{'layer_name': 'annotation', 'type': 'line', 'point': ([257872.09375, 177939.875, 2646.5], [257974.09375, 178158.875, 2646.5])}
{'layer_name': 'annotation', 'type': 'ellipsoid', 'point': ([257797.46875, 178220.9375, 2646.5], [58, 75, 0])}


In [23]:
annotations_df

,layer_name,type,point
0,annotation,line,"257872.09375, 177939.875, 2646.5 - 257974.0937..."
1,annotation,ellipsoid,"257797.46875, 178220.9375, 2646.5 - 58, 75, 0"
